In [151]:
import re
import math

In [235]:
import requests
import json

HOST = "http://10.35.0.112:8000/"
API_CANCLED = "canclled"
API_PNR_RANKING = "pnr_ranking?flight_id="
API_ALT_FLIGHT="alt_flight?flight_id="

def get_cancled():
    response = requests.request("GET", HOST + API_CANCLED, headers={}, data={})
    return json.loads(response.text)

def get_pnrs(flight_id):
    response = requests.request("GET", HOST + API_PNR_RANKING + str(flight_id), headers={}, data={})
    return json.loads(response.text)

def get_alt_flights(flight_id):
    response = requests.request("GET", HOST + API_ALT_FLIGHT + str(flight_id), headers={}, data={})
    return json.loads(response.text)

In [254]:
cancel=get_cancled()
len(cancel)
x=get_alt_flights('61d83160')
x

{'data': {'data': [{'flight_id': '24565385',
    'departure_airport': 'TJV',
    'arrival_airport': 'GBI',
    'status': 'Scheduled',
    'arrival_time': '2023-12-14T08:19:29.683Z',
    'departure_time': '2023-12-14T06:42:29.683Z',
    'total_avilable_seats': 4,
    'delay': 'P0DT02H22M59.597217S',
    'flight_time': 'P0DT01H37M00S',
    'avilable_seats': {'F': 0,
     'P': 0,
     'C': 1,
     'J': 0,
     'Z': 0,
     'Q': 0,
     'R': 0,
     'S': 0,
     'T': 0,
     'H': 0,
     'M': 0,
     'Y': 0,
     'A': 0,
     'B': 0,
     'D': 0,
     'G': 0,
     'I': 0,
     'E': 0,
     'K': 0,
     'L': 0,
     'N': 0,
     'O': 2,
     'U': 0,
     'V': 0,
     'W': 0,
     'X': 1}},
   {'flight_id': '25a29308',
    'departure_airport': 'CJB',
    'arrival_airport': 'MYQ',
    'status': 'Scheduled',
    'arrival_time': '2023-12-15T12:59:30.314Z',
    'departure_time': '2023-12-15T10:00:30.314Z',
    'total_avilable_seats': 81,
    'delay': 'P1DT05H41M00.228400S',
    'flight_time': 'P

In [252]:
map_cabin={ 'F':'F','P':'F','C':'B','J':'B','Z':'B','Q':'P','R':'P','S':'P','T':'P','H':'P','M':'P',
          'Y':'E', 'A':'E', 'B':'E', 'D':'E', 'E':'E', 'G':'E', 'I':'E', 'K':'E', 'L':'E', 'N':'E', 'O':'E',
           'U':'E', 'V':'E','W':'E','X':'E'}
cabins=['F','B','P','E']
cabin_ind={'F':0 , 'B':1,'P':2,'E':3}

class_ind = {'F':'0', 'P':'1', 'C':'2', 'J':'3', 'Z':'4', 'Q':'5', 'R':'6', 'S':'7', 'T':'8', 'H':'9', 'M':'10',
             'Y':'11', 'A':'12', 'B':'13', 'D':'14', 'E':'15', 'G':'16', 'I':'17', 'K':'18', 'L':'19', 'N':'20', 'O':'21',
             'U':'22', 'V':'23', 'W':'24', 'X':'25'}

ind_class = {'0': 'F', '1': 'P', '2': 'C', '3': 'J', '4': 'Z', '5': 'Q', '6': 'R', '7': 'S', '8': 'T', '9': 'H', '10': 'M',
             '11': 'Y', '12': 'A', '13': 'B', '14': 'D', '15': 'E', '16': 'G', '17': 'I', '18': 'K', '19': 'L', '20': 'N', '21': 'O',
            '22': 'U', '23': 'V', '24': 'W', '25': 'X'}

In [366]:
pnr={}
alt_flight={}
# pnr_flight={}
for f in cancel['data']:
#     print(f['flight_id'])
    pnr[f['flight_id']]=get_pnrs(f['flight_id'])['data']
    alt=get_alt_flights(f['flight_id'])['data']
    f2=[]
    for flight in alt['data']:
        f3={}
        f3['flight_id']=flight['flight_id']
        f3['delay']=flight['delay']
        f3['flight_time']=flight['flight_time']
        f3['F']=0
        f3['B']=0
        f3['P']=0
        f3['E']=0
#         print()
        for key in map_cabin:
            if key in flight['avilable_seats']:
                f3[map_cabin[key]]+=flight['avilable_seats'][key]
        f2.append(f3)
    f4=[]
    for flight in alt['c_flights']:
        f5=[]
        for fl in flight:
            f3={}
            f3['flight_id']=fl['flight_id']
            f3['delay']=fl['delay']
            f3['flight_time']=fl['flight_time']
            f3['F']=0
            f3['B']=0
            f3['P']=0
            f3['E']=0
#         print()
            for key in map_cabin:
                if key in fl['avilable_seats']:
                    f3[map_cabin[key]]+=fl['avilable_seats'][key]
            f5.append(f3)
        f4.append(f5)
#     print(f['flight_id'])
    alt_flight[f['flight_id']]={'n_flights':f2,'c_flights':f4}

In [367]:
# for k in alt_flight:
#     print(k)

In [368]:
used_seat={}
max_seat={}

for i in alt_flight:
    for j in alt_flight[i]['n_flights']:
        used_seat[j['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
    for j in alt_flight[i]['c_flights']:
        used_seat[j[0]['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
        used_seat[j[1]['flight_id']]={ 'F':'0','P':'0','C':'0','J':'0','Z':'0','Q':'0','R':'0','S':'0','T':'0','H':'0','M':'0',
          'Y':'0', 'A':'0', 'B':'0', 'D':'0', 'E':'0', 'G':'0', 'I':'0', 'K':'0', 'L':'0', 'N':'0', 'O':'0',
           'U':'0', 'V':'0','W':'0','X':'0'}
        

In [509]:
used_seat={}
maximum_seat={}

for i in alt_flight:
    for j in alt_flight[i]['n_flights']:
        used_seat[j['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j['flight_id']]={'F':j['F'],'B':j['B'],'P':j['P'],'E':j['E']}
    for j in alt_flight[i]['c_flights']:
        used_seat[j[0]['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j[0]['flight_id']]={'F':j[0]['F'],'B':j[0]['B'],'P':j[0]['P'],'E':j[0]['E']}
        used_seat[j[1]['flight_id']]={'F':0,'B':0,'P':0,'E':0}
        maximum_seat[j[1]['flight_id']]={'F':j[1]['F'],'B':j[1]['B'],'P':j[1]['P'],'E':j[1]['E']}
        

In [510]:
used_seat

{'49275e19': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'f02572c1': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '789d46bd': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '241235aa': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '01787d3c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '9608a301': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '964283f2': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '2d4bb5d8': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '2513e2bd': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '3770b666': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'aed6747a': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '4772908c': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'ce729d21': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '2d190194': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '5de5ca13': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'ee7a82d9': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '342c63c4': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '9b9f8d16': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'cd623fb0': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '868de684': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 'cfffe04f': {'F': 0, 'B': 0, 'P': 0, 'E': 0},
 '754244c1': 

In [511]:
needed_seat=[]
for i in pnr:
    alloc={}
    for j in pnr[i]:
        alloc=j;
        alloc['flight_id']=i
        needed_seat.append(alloc)
sorted_pnr = sorted(needed_seat, key=lambda x: x['score'],reverse=True)

In [512]:
allocated={}

for i in sorted_pnr:
    allocated[i['pnr']]='NULL'

In [513]:
alt_flight

{'434c32f8': {'n_flights': [{'flight_id': '49275e19',
    'delay': 'P0DT19H59M00.288393S',
    'flight_time': 'P0DT01H34M00S',
    'F': 7,
    'B': 0,
    'P': 3,
    'E': 18},
   {'flight_id': 'f02572c1',
    'delay': 'P1DT02H38M00.121958S',
    'flight_time': 'P0DT00H18M00S',
    'F': 1,
    'B': 1,
    'P': 1,
    'E': 3}],
  'c_flights': []},
 'a715ea42': {'n_flights': [], 'c_flights': []},
 'a0a4aad2': {'n_flights': [{'flight_id': '789d46bd',
    'delay': 'P1DT04H37M00.202020S',
    'flight_time': 'P0DT00H58M00S',
    'F': 0,
    'B': 8,
    'P': 8,
    'E': 23},
   {'flight_id': '241235aa',
    'delay': 'P1DT20H39M00.274306S',
    'flight_time': 'P0DT02H35M00S',
    'F': 6,
    'B': 18,
    'P': 15,
    'E': 32}],
  'c_flights': [[{'flight_id': '01787d3c',
     'delay': 'P1DT11H45M00.524727S',
     'flight_time': 'P0DT00H41M00S',
     'F': 9,
     'B': 15,
     'P': 17,
     'E': 28},
    {'flight_id': '9608a301',
     'delay': 'P0DT04H02M00.172526S',
     'flight_time': 'P0DT00H

In [514]:
def get_delay_cost(delay_str):
    pattern = re.compile(r'P(\d+)DT(\d+)H(\d+)M')
    match = pattern.match(delay_str)
    days, hours, minutes = map(int, match.groups())
    time = days*24 + hours
    if(minutes>30):
        time+=1
    time = math.exp(time/3)
    return time

def get_layoff_cost(delay_str):
    pattern = re.compile(r'P(\d+)DT(\d+)H(\d+)M')
    match = pattern.match(delay_str)
    days, hours, minutes = map(int, match.groups())
    time = days*24 + hours
    if(minutes>30):
        time+=1
    time = math.exp(time/2.5)
    return time


def get_flight_time_score(d,s):
    pattern = re.compile(r'P(\d+)DT(\d+)H(\d+)M')
    match = pattern.match(s)
    days, hours, minutes = map(int, match.groups())
    time = days*24 + hours
    if(minutes>30):
        time+=1
    if d==0 :
        return 0
    elif(d>0):
        return math.exp(time/1.2)
    else:
        return math.exp(time/1.5)

    

In [515]:
tot_cost=0

In [516]:
tot_cost=0
def obj(n_flights,c_flights,pnr):
    global tot_cost
#     c_flights=[]
#     print(n_flights[0])
#     print(c_flights[0])
#     return
#     print(len(n_flights))
#     print(len(c_flights))
#     print(n_flights)
    f_id=[-1]
    c_id=[-1]
    val=1e18
    c_up=3
    c_down=2
    temp_id=0
    cabin0=cabin_ind[map_cabin[pnr['class']]]
    for f in  n_flights:
#         print(f)
#         break
        cost=get_delay_cost(f['delay'])
            
        for i in cabins:
            if(f[i]<pnr['pax']+used_seat[f['flight_id']][i]):
                continue
            d=cabin_ind[i]-cabin0
            c=get_flight_time_score(d,f['flight_time'])
#             print(c+cost)
            if(c+cost<val):
                val=c+cost
                f_id=[f['flight_id']]
                c_id=[i]
#             temp_id+=1
#     flight_no=temp_id/n_class
#     class_no=temp_id%n_class
#     print('kkkkkk')
    for f in c_flights:
        cost=get_delay_cost(f[0]['delay'])+get_layoff_cost(f[1]['delay'])
        for i in cabins:
            for j in cabins:
                if(f[0][i]<pnr['pax']+used_seat[f[0]['flight_id']][i] or f[1][j]<pnr['pax']+used_seat[f[0]['flight_id']][j]):
                    continue
                d1=cabin_ind[i]-cabin0
                d2=cabin_ind[j]-cabin0
                c=get_flight_time_score(d1,f[0]['flight_time'])
                c+=get_flight_time_score(d2,f[1]['flight_time'])
#                 print(c+cost)
                if(c+cost<val):
                    val=c+cost
                    f_id=[f[0]['flight_id'],f[1]['flight_id']]
                    c_id=[i,j]
    if val!=1e18:
        tot_cost+=val
    return f_id,c_id,val

In [517]:
# tot_cost=0
for i in sorted_pnr:
#     print(i)
    f_id=i['flight_id']
    flight_id,cabin_id,cost=obj(alt_flight[f_id]['n_flights'],alt_flight[f_id]['c_flights'],i)
#     break
#     print(flight_id)
#     print(cabin_id)
    if(len(flight_id)==2):
        allocated[i['pnr']]=[flight_id,cabin_id,cost]
        used_seat[flight_id[0]][cabin_id[0]]+=i['pax']
        used_seat[flight_id[1]][cabin_id[1]]+=i['pax']
    elif(flight_id[0]!=-1):
        allocated[i['pnr']]=[flight_id[0],cabin_id[0],cost]
        used_seat[flight_id[0]][cabin_id[0]]+=i['pax']

In [518]:
 allocated

{'8e4a17': ['00d447fe', 'F', 14.391916095149892],
 'f87a3d': ['2d190194', 'E', 13.578106385789562],
 '6f8d9c': ['24565385', 'B', 1.9477340410546757],
 'c65e4d': ['2d4bb5d8', 'B', 12.68354614940068],
 '5fc3e8': ['b6762338', 'P', 1.0],
 '895ee8': ['10e3f71a', 'B', 42902.69723267145],
 '105e86': ['28cf5c8d', 'B', 15.606748551795796],
 '83b0d7': 'NULL',
 '7b9876': ['219da628', 'E', 15.606748551795796],
 'e6a0fd': ['1f1dee23', 'E', 30.332600785418965],
 'c40f05': ['67b0beb4', 'E', 227147.90363670734],
 'ce950f': ['14cecff1', 'F', 14.391916095149892],
 '38ab49': ['ecd5012a', 'F', 5.29449005047003],
 '2383b0': 'NULL',
 'fccab6': [['5c84b850', 'cbad8119'], ['B', 'F'], 177.1085675518791],
 'd127d8': ['cf529ca7', 'E', 83563.39709498636],
 'c3e5e0': 'NULL',
 '50e663': ['3e2b52b9', 'F', 1.9477340410546757],
 '4a387b': ['3e2b52b9', 'F', 1.9477340410546757],
 '58e903': ['3e2b52b9', 'F', 1.9477340410546757],
 'd82291': ['847b2dbb', 'E', 9.088157945153206],
 '49939c': ['847b2dbb', 'E', 9.0881579451532

{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'F': 0, '

In [508]:
pnr_flight={}
for i in sorted_pnr:
    pnr_flight[i['pnr']]=i['flight_id']

flight_cancel

{'54249011': 123,
 '22405d17': 114,
 'a715ea42': 119,
 'ea9b3e04': 99,
 '9bd2ef77': 93,
 '51a72338': 86,
 'a3272bac': 89,
 '95f0a4df': 117,
 '9f8f0734': 116,
 'de106cb7': 35,
 '11e64a79': 34,
 '4934ca7a': 126,
 '1d0decf2': 68,
 '40fdf34a': 81,
 '5ec776c4': 100,
 '4d898062': 100,
 '8f04e628': 24,
 'd816a431': 37,
 'e0522dcf': 71,
 '9e254b38': 41,
 '402034ca': 49,
 '8c662383': 62,
 '765974c3': 23,
 '8fc3fb19': 67,
 'fcf953ec': 39,
 '380048b0': 16,
 '61d83160': 22,
 '2c9bc381': 31,
 '8647babf': 26,
 'cf6b7de1': 29,
 '9bf1e4a8': 19,
 '434c32f8': 9,
 'e718fb26': 5,
 '7e082096': 14}

In [505]:
len(flight_cancel)

34

In [507]:
remaining_seat={}
for i in used_seat:
    k={}
    for j in used_seat[i]:
        k[j]=maximum_seat[i][j]-used_seat[i][j]
    remaining_seat[i]=k 
remaining_seat
sum=0
flight_cancel={}
for i in allocated: 
    if allocated[i]=='NULL':
        if pnr_flight[i] in flight_cancel:
            flight_cancel[pnr_flight[i]]+=1
        else:
            flight_cancel[pnr_flight[i]]=1 
        for k in alt_flight[pnr_flight[i]]['n_flights']:
            print(k)
            print(remaining_seat[k['flight_id']])

{'flight_id': '1f1dee23', 'delay': 'P0DT10H10M59.649790S', 'flight_time': 'P0DT00H40M00S', 'F': 2, 'B': 3, 'P': 2, 'E': 6}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'flight_id': 'cf529ca7', 'delay': 'P1DT10H10M59.649790S', 'flight_time': 'P0DT00H40M00S', 'F': 0, 'B': 3, 'P': 2, 'E': 6}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'flight_id': '2d190194', 'delay': 'P0DT01H08M59.924950S', 'flight_time': 'P0DT02H59M00S', 'F': 0, 'B': 2, 'P': 2, 'E': 3}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'flight_id': 'd1947ed7', 'delay': 'P0DT14H42M59.436976S', 'flight_time': 'P0DT01H25M00S', 'F': 1, 'B': 0, 'P': 3, 'E': 4}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'flight_id': '2d190194', 'delay': 'P0DT01H08M59.924950S', 'flight_time': 'P0DT02H59M00S', 'F': 0, 'B': 2, 'P': 2, 'E': 3}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'flight_id': '754244c1', 'delay': 'P0DT13H19M59.803388S', 'flight_time': 'P0DT01H08M00S', 'F': 1, 'B': 1, 'P': 2, 'E': 5}
{'F': 0, 'B': 0, 'P': 0, 'E': 0}
{'flight_id': '61ff4c1d', 'delay': 'P1DT13H19M59.803388S', 'flig

In [495]:
maximum_seat

{'49275e19': {'F': 7, 'B': 0, 'P': 3, 'E': 18},
 'f02572c1': {'F': 1, 'B': 1, 'P': 1, 'E': 3},
 '789d46bd': {'F': 0, 'B': 8, 'P': 8, 'E': 23},
 '241235aa': {'F': 6, 'B': 18, 'P': 15, 'E': 32},
 '01787d3c': {'F': 9, 'B': 15, 'P': 17, 'E': 28},
 '9608a301': {'F': 9, 'B': 9, 'P': 15, 'E': 38},
 '964283f2': {'F': 0, 'B': 2, 'P': 2, 'E': 3},
 '2d4bb5d8': {'F': 3, 'B': 11, 'P': 10, 'E': 31},
 '2513e2bd': {'F': 0, 'B': 3, 'P': 4, 'E': 9},
 '3770b666': {'F': 4, 'B': 12, 'P': 5, 'E': 24},
 'aed6747a': {'F': 0, 'B': 2, 'P': 4, 'E': 4},
 '4772908c': {'F': 12, 'B': 13, 'P': 18, 'E': 31},
 'ce729d21': {'F': 2, 'B': 2, 'P': 4, 'E': 5},
 '2d190194': {'F': 0, 'B': 2, 'P': 2, 'E': 3},
 '5de5ca13': {'F': 0, 'B': 5, 'P': 13, 'E': 45},
 'ee7a82d9': {'F': 8, 'B': 14, 'P': 11, 'E': 18},
 '342c63c4': {'F': 7, 'B': 9, 'P': 15, 'E': 47},
 '9b9f8d16': {'F': 14, 'B': 24, 'P': 9, 'E': 31},
 'cd623fb0': {'F': 1, 'B': 5, 'P': 2, 'E': 6},
 '868de684': {'F': 1, 'B': 2, 'P': 0, 'E': 3},
 'cfffe04f': {'F': 6, 'B': 17, 

In [490]:
sum

2084

In [491]:
x=len(allocated)-sum
x

1930

In [492]:
tot_cost/x

187056.43665020113

In [493]:
tot_cost

361018922.7348882